<a href="https://colab.research.google.com/github/rlrocha90/TP555_projeto_final/blob/main/GTR_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Global Tracking Transformers

<img align="center" src="https://github.com/xingyizhou/GTR/raw/master/docs/GTR_teaser.jpg" width="800">

This is a colab demo of using GTR (**G**lobal **Tr**acking Transformers). We will use the pretrained GTR models to run global tracking on an example video.

This demo is modified from the [detectron2 colab tutorial](https://colab.research.google.com/drive/16jcaJoc6bCFAQ96jDe2HwtXj7BMD_-m5).

You can make a copy of this tutorial by "File -> Open in playground mode" and make changes there. __DO NOT__ request access to this tutorial.


In [ ]:
# Install detectron2
import torch
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
# Install detectron2 that matches the above pytorch version
# See https://detectron2.readthedocs.io/tutorials/install.html for instructions
# !pip install detectron2==0.6 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

torch:  1.11 ; cuda:  cu113
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-ovnb6376
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-ovnb6376


In [ ]:
# clone and install GTR
!git clone https://github.com/xingyizhou/GTR.git --recurse-submodules
%cd GTR
!pip install -r requirements.txt

fatal: destination path 'GTR' already exists and is not an empty directory.
/content/GTR
  Cloning https://github.com/TAO-Dataset/tao to /tmp/pip-req-build-5dqa6qvu
  Running command git clone -q https://github.com/TAO-Dataset/tao /tmp/pip-req-build-5dqa6qvu
  Cloning https://github.com/achalddave/python-script-utils.git (to revision v0.0.2) to /tmp/pip-install-plan2zs3/script-utils_f4810b70184e4634963d35b557747a2b
  Running command git clone -q https://github.com/achalddave/python-script-utils.git /tmp/pip-install-plan2zs3/script-utils_f4810b70184e4634963d35b557747a2b
  Running command git checkout -q 26dcd3b0c3487f46d31b4997c824e43a1ec5c897
  Running command git submodule update --init --recursive -q


In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import sys
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# GTR libraries
sys.path.insert(0, 'third_party/CenterNet2/')
from centernet.config import add_centernet_config
from gtr.config import add_gtr_config
from gtr.predictor import GTRPredictor, TrackingVisualizer

In [ ]:
# Download models
import gdown
!mkdir models/
gdown.download("https://drive.google.com/u/1/uc?id=1TqkLpFZvOMY5HTTaAWz25RxtLHdzQ-CD", "models/GTR_TAO_DR2101.pth")

mkdir: cannot create directory ‘models/’: File exists


Downloading...
From: https://drive.google.com/u/1/uc?id=1TqkLpFZvOMY5HTTaAWz25RxtLHdzQ-CD
To: /content/GTR/models/GTR_TAO_DR2101.pth
100%|██████████| 512M/512M [00:04<00:00, 119MB/s]


'models/GTR_TAO_DR2101.pth'

In [ ]:
# Build the detector and download our pretrained weights
cfg = get_cfg()
add_centernet_config(cfg)
add_gtr_config(cfg)
cfg.merge_from_file("configs/GTR_TAO_DR2101.yaml")
cfg.MODEL.WEIGHTS = 'models/GTR_TAO_DR2101.pth'
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# cfg.MODEL.DEVICE='cpu' # uncomment this to use cpu-only mode.
metadata = MetadataCatalog.get(
    cfg.DATASETS.TEST[0] if len(cfg.DATASETS.TEST) else "__unused")
predictor = GTRPredictor(cfg)
tracker_visualizer = TrackingVisualizer(metadata)

WARNING [04/25 17:18:44 d2.config.compat]: Config 'configs/GTR_TAO_DR2101.yaml' has no VERSION. Assuming it to be compatible with latest v2.


Some model parameters or buffers are not found in the checkpoint:
roi_heads.box_predictor.0.{fed_loss_cls_weights, freq_weight}
roi_heads.box_predictor.1.{fed_loss_cls_weights, freq_weight}
roi_heads.box_predictor.2.{fed_loss_cls_weights, freq_weight}
The checkpoint state_dict contains keys that are not used by the model:
  roi_heads.pos_emb.weight


In [ ]:
# Functions to load and same videos
import imageio
from IPython.core.display import Video
from IPython.display import display
def show_video(filename, frames, fps=5):
    imageio.mimwrite(
        filename, [x[..., ::-1] for x in frames], fps=fps)
    display(Video(filename, embed=True))

def _frame_from_video(video):
    while video.isOpened():
        success, frame = video.read()
        if success:
            yield frame
        else:
            break

In [ ]:
# Load images from video
video_path = 'docs/yfcc_v_acef1cb6d38c2beab6e69e266e234f.mp4'
video = cv2.VideoCapture(video_path)
width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
frames_per_second = video.get(cv2.CAP_PROP_FPS)
num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
basename = os.path.basename(video_path)
codec, file_ext = "mp4v", ".mp4"
# codec = 'H264'
frames = [x for x in _frame_from_video(video)]
video.release()

show_video('input.mp4', frames)

In [ ]:
# Run model
outputs = predictor(frames)

/usr/local/lib/python3.7/dist-packages/torch/functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [ ]:
# Post processing and save output video
def _process_predictions(tracker_visualizer, frame, predictions):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    predictions = predictions["instances"].to('cpu')
    vis_frame = tracker_visualizer.draw_instance_predictions(
        frame, predictions)
    vis_frame = cv2.cvtColor(vis_frame.get_image(), cv2.COLOR_RGB2BGR)
    return vis_frame

out_frames = []
for frame, instances in zip(frames, outputs):
    out_frame = _process_predictions(tracker_visualizer, frame, instances)
    out_frames.append(out_frame)

show_video('output.mp4', out_frames)

In [ ]:
# Download results
# from google.colab import files
# files.download('output.mp4')